In [1]:
import datasets
from datasets import load_dataset
from IPython.display import display, Markdown

dataset = load_dataset('deepmind/narrativeqa')
dataset

df = dataset['validation'].to_pandas()

/home/ubuntu/1xa100-2/llama-stack/envs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df['context'] = df['document'].apply(lambda x: x['text'])
df['question'] = df['question'].apply(lambda x: x['text'])
df = df.rename(columns={'question': 'questions'})
df = df.groupby('context').agg({'questions': list}).reset_index()
# Get first 50 rows
df = df.iloc[:50]
df.shape

(50, 2)

In [3]:
%pip install docling


  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached torchvision-0.20.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached ninja-1.11.1.2-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 142.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 60.2 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 158.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 44.9 MB/s eta 0:00:00
Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 141.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from docling.document_converter import DocumentConverter, DocumentStream
from io import BytesIO

converter = DocumentConverter()

# Get first document text and questions
doc_text = df.iloc[0]['context']

# Convert document
buf = BytesIO(doc_text.encode('utf-8'))
source = DocumentStream(name="my_doc.pdf", stream=buf)
doc = converter.convert(source)
# Write document to output.txt
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write(doc.document.export_to_text())